In [1]:
import pyspark

### 注意
以这种方式启动spark,会自动启动一个SparkContext,记为sc; <br>
只能有一个SparkContext,想要自己开启,需要先sc.stop(),然后再启动新的SparkContext

In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
sc.stop()
sc = pyspark.SparkContext("local", "WordCount").getOrCreate()

In [4]:
sc

<SparkContext master=local appName=WordCount>

In [5]:
text = sc.textFile("file:///home/liuyang/license.txt")
print(text)

file:///home/liuyang/license.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [6]:
text.collect()

['Copyright (c) 2000-2005 INRIA, France Telecom',
 'All rights reserved.',
 '',
 'Redistribution and use in source and binary forms, with or without',
 'modification, are permitted provided that the following conditions',
 'are met:',
 '',
 '1. Redistributions of source code must retain the above copyright',
 '   notice, this list of conditions and the following disclaimer.',
 '',
 '2. Redistributions in binary form must reproduce the above copyright',
 '   notice, this list of conditions and the following disclaimer in the',
 '   documentation and/or other materials provided with the distribution.',
 '',
 '3. Neither the name of the copyright holders nor the names of its',
 '   contributors may be used to endorse or promote products derived from',
 '   this software without specific prior written permission.',
 '',
 'THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"',
 'AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE',
 'IMPLIED WARR

### Spark WordCount

In [7]:
counts = text.flatMap(lambda line: line.lower().strip().split()) \
             .filter(lambda line: line) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [8]:
counts.collect()

[('copyright', 6),
 ('(c)', 1),
 ('2000-2005', 1),
 ('inria,', 1),
 ('france', 1),
 ('telecom', 1),
 ('all', 1),
 ('rights', 1),
 ('reserved.', 1),
 ('redistribution', 1),
 ('and', 8),
 ('use', 2),
 ('in', 6),
 ('source', 2),
 ('binary', 2),
 ('forms,', 1),
 ('with', 2),
 ('or', 10),
 ('without', 2),
 ('modification,', 1),
 ('are', 3),
 ('permitted', 1),
 ('provided', 3),
 ('that', 1),
 ('the', 15),
 ('following', 3),
 ('conditions', 3),
 ('met:', 1),
 ('1.', 1),
 ('redistributions', 2),
 ('of', 13),
 ('code', 1),
 ('must', 2),
 ('retain', 1),
 ('above', 2),
 ('notice,', 2),
 ('this', 5),
 ('list', 2),
 ('disclaimer.', 1),
 ('2.', 1),
 ('form', 1),
 ('reproduce', 1),
 ('disclaimer', 1),
 ('documentation', 1),
 ('and/or', 1),
 ('other', 1),
 ('materials', 1),
 ('distribution.', 1),
 ('3.', 1),
 ('neither', 1),
 ('name', 1),
 ('holders', 2),
 ('nor', 1),
 ('names', 1),
 ('its', 1),
 ('contributors', 3),
 ('may', 1),
 ('be', 2),
 ('used', 1),
 ('to', 1),
 ('endorse', 1),
 ('promote', 1),


### RDD的转化操作和执行操作

In [9]:
rdd = sc.parallelize([[1], [1], [2, 3]])

In [10]:
rdd

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:194

In [11]:
map_rdd = rdd.map(lambda x: [sum(x) + 1, 1])

# Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item). 
flat_map_rdd = rdd.flatMap(lambda x: [sum(x) + 1, 1])

filter_rdd = rdd.filter(lambda x: x != 1)
distinct_rdd = flat_map_rdd.distinct()
sample_rdd = rdd.sample(True, 0.7)

In [12]:
print(map_rdd)
print(flat_map_rdd)
print(filter_rdd)
print(distinct_rdd)
print(sample_rdd)

PythonRDD[12] at RDD at PythonRDD.scala:52
PythonRDD[13] at RDD at PythonRDD.scala:52
PythonRDD[14] at RDD at PythonRDD.scala:52
PythonRDD[15] at RDD at PythonRDD.scala:52
PythonRDD[16] at RDD at PythonRDD.scala:52


In [13]:
print(map_rdd.collect())
print(flat_map_rdd.collect())
print(filter_rdd.collect())
print(sample_rdd.collect())

[[2, 1], [2, 1], [6, 1]]
[2, 1, 2, 1, 6, 1]
[[1], [1], [2, 3]]
[[2, 3]]


### Logistic Regression
可以在一个 Spark应用中使用多个SparkSession,这样子就可以通过SparkSession将多个关系实体隔离开来

In [14]:
from pyspark.ml.classification import LogisticRegression

spark = pyspark.sql.SparkSession.builder.appName("Logistic Regression").getOrCreate()
spark

In [15]:
training = spark.read.format("libsvm").load("file:///home/liuyang/libsvm_data.txt")
training.head(3)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

In [16]:
lr = LogisticRegression(maxIter=10, 
                        regParam=0.3,  # regularization parameter
                        elasticNetParam=0.8) # ElasticNet mixing parameter
lrModel = lr.fit(training)

In [17]:
print(f'intercept: {lrModel.intercept}')
print(f'coeffs: {lrModel.coefficients}')

intercept: 0.22456315961250325
coeffs: (692,[244,263,272,300,301,328,350,351,378,379,405,406,407,428,433,434,455,456,461,462,483,484,489,490,496,511,512,517,539,540,568],[-7.353983524188197e-05,-9.102738505589466e-05,-0.00019467430546904298,-0.00020300642473486668,-3.1476183314863995e-05,-6.842977602660743e-05,1.5883626898239883e-05,1.4023497091372047e-05,0.00035432047524968605,0.00011443272898171087,0.00010016712383666666,0.0006014109303795481,0.0002840248179122762,-0.00011541084736508837,0.000385996886312906,0.000635019557424107,-0.00011506412384575676,-0.00015271865864986808,0.0002804933808994214,0.0006070117471191634,-0.0002008459663247437,-0.0001421075579290126,0.0002739010341160883,0.00027730456244968115,-9.838027027269332e-05,-0.0003808522443517704,-0.00025315198008555033,0.00027747714770754307,-0.0002443619763919199,-0.0015394744687597765,-0.00023073328411331293])


#### 模型迭代细节

In [18]:
objectiveHistory = lrModel.summary.objectiveHistory

iIter = 0
print("objectiveHistory:")
for objective in objectiveHistory:
    iIter += 1
    print(f"Iter-{iIter}: {objective}")

objectiveHistory:
Iter-1: 0.6833149135741672
Iter-2: 0.6662875751473734
Iter-3: 0.6217068546034618
Iter-4: 0.6127265245887887
Iter-5: 0.6060347986802873
Iter-6: 0.6031750687571562
Iter-7: 0.5969621534836274
Iter-8: 0.5940743031983118
Iter-9: 0.5906089243339022
Iter-10: 0.5894724576491042
Iter-11: 0.5882187775729587


In [19]:
preds = lrModel.transform(training)
print(preds.take(1))

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0